In [1]:
from selenium import webdriver
import requests
import requests_cache
from bs4 import BeautifulSoup
import time
import json
import re
import pandas as pd 
from datetime import datetime, timedelta 
from datetime import date
import urllib.robotparser 
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

In [2]:
# YOUR CODE HERE
def denied(response):
    if "\"incomplete_results\":true" in response.text:
        return False
    else:
        return True
requests_cache.install_cache(expire_after = None, filter_fn = denied)

In [3]:
url = "https://soundcloud.com/search/people?q=*"
service = Service("chromedriver.exe")
#cái này setup theo mấy trang nó chỉ!!
driver1 = webdriver.Chrome(service = service)
driver1.get(url)
time.sleep(1)
soup = BeautifulSoup(driver1.page_source)
cookies_class = soup.find("button",{"id":"onetrust-accept-btn-handler"})["id"]
cookies_click = driver1.find_element(By.XPATH,"//button[@id=\'" + cookies_class + "\']")
cookies_click.click()

driver2 = webdriver.Chrome(service = service)
driver2.get(url)
time.sleep(1)
soup = BeautifulSoup(driver2.page_source)
cookies_class = soup.find("button",{"id":"onetrust-accept-btn-handler"})["id"]
cookies_click = driver2.find_element(By.XPATH,"//button[@id=\'" + cookies_class + "\']")
cookies_click.click()

In [4]:
def write_file(page_source):    
    soup = BeautifulSoup(page_source)
    sound_item = soup.find_all("li",{"class","soundList__item"})

    count_item = 0
    for item in sound_item:
        if count_item == 6:
            break
        count_item += 1
        item_url = item.find("a",{"class","sound__coverArt"})["href"]
#             print(item_url)
        sound_url = "https://soundcloud.com/" + item_url
        req = requests.get(sound_url)
        soup = BeautifulSoup(req.text, "lxml")
        search = soup.find_all("script",{"":""})
        for script in search:
            if "window.__sc_hydration" in script.text:
                search = script
                break
#         print(search)
        res = json.loads(search.text[24:-1])
        hydratable_playlist = res[6]["data"]
#             print(hydratable_playlist)
        for i in hydratable_playlist.keys():
            if hydratable_playlist[i] == "":
                hydratable_playlist[i] = "None"

        for attr in attributes:
            if attr == "tracks":
                break
            output.write(str(hydratable_playlist[attr]).replace(",",";").replace("\n","/"))
            output.write(",")
            
        trackid_list = ""
        for track in hydratable_playlist["tracks"]:
            trackid_list = trackid_list + str(track["id"]) + ";"
        output.write(trackid_list)
        output.write("\n")

In [5]:
users_urls = []
users_urls_file = "good_user.txt"
with open(users_urls_file) as f:
    for line in f:
        users_urls.append(line.replace("\n",""))
        
attributes = ["artwork_url","created_at","duration","embeddable_by"
              ,"genre","id","kind","label_name","last_modified","license"
              ,"likes_count","managed_by_feeds","permalink_url","release_date"
              ,"purchase_title","purchase_url","reposts_count","sharing"
              ,"tag_list","title","user_id","is_album"
              ,"published_at","display_date","track_count","tracks"]
output = open("playlist.csv","w",encoding = "utf-8")

for attr in attributes:
    output.write(attr)
    output.write(",")
output.write("\n")
step = 0
users_urls.append("")

driver1.get(users_urls[0]+"/sets")
time.sleep(1.5)
print("total:",len(users_urls))
for url in users_urls:
    try:
        step+=1
        if step % 100==0:
            print("%",step)
#             break
        else:
            if step %10==0:
                print("#",end="")
#                 break

        if step == 1: 
            continue
        
        if url=="":
            if i%2==0:
                write_file(driver1.page_source)
            else:
                write_file(driver2.page_source)
        
        if step%2==0:
            driver2.get(url+"/sets")
            write_file(driver1.page_source)
        else:
            driver1.get(url+"/sets")
            write_file(driver2.page_source)
#         time.sleep(1.5)
    except:
        continue
    
print("finish")
output.close()

total: 2571
#########% 100
#########% 200
#########% 300
#########% 400
#########% 500
#########% 600
#########% 700
#########% 800
#########% 900
#########% 1000
#########% 1100
#########% 1200
#########% 1300
#########% 1400
#########% 1500
#########% 1600
#########% 1700
#########% 1800
#########% 1900
#########% 2000
#########% 2100
#########% 2200
#########% 2300
#########% 2400
#########% 2500
#######finish
